In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.interpolation import rotate
import pickle

from collections import defaultdict
import networkx as nx
from scipy.linalg import cholesky
from data.utils import *

In [ ]:
with open('data/data.pickle', 'rb') as f:
    data_new = pickle.load(f)
    
env = data_new['env']
obj = data_new['obj']

In [ ]:
plt.imshow(env)
# print(np.unique(env, return_counts=True))
print(env.shape)
print(env[300,10])
print(is_state_observable(env,(300,10), 1))

In [ ]:
plt.imshow(obj)

# GRAPH RRBT

In [154]:
from dubins import connect
from propagate import DynamicModel,BeliefNode

def distance_angle_all(point1, point2):
    return np.linalg.norm(point1[:, :2] - point2[:2], axis=1)[:, None] \
                + 20 * angle_difference(point1[:, -1], finish_point[-1])[:, None]

def uncertainty_collision_check(env,mu, Sigma, nSigma = 1,n_points = 50):
    """
    Plots a 2D covariance ellipse given the Gaussian distribution parameters.
    The function expects the mean and covariance matrix to ignore the theta parameter.

    :param mu: The mean of the distribution: 2x1 vector.
    :param Sigma: The covariance of the distribution: 2x2 matrix.
    :param color: The border color of the ellipse and of the major and minor axes.
    :param nSigma: The radius of the ellipse in terms of the number of standard deviations (default: 1).
    :param legend: If not None, a legend label to the ellipse will be added to the plot as an attribute.
    """
    mu = np.array(mu)
    assert mu.shape == (2,)
    Sigma = np.array(Sigma)
    assert Sigma.shape == (2, 2)

    A = cholesky(Sigma, lower=True)

    angles = np.linspace(0, 2 * np.pi, n_points)
    x_old = nSigma * np.cos(angles)
    y_old = nSigma * np.sin(angles)

    x_y_old = np.stack((x_old, y_old), 1)
    x_y_new = np.matmul(x_y_old, np.transpose(A)) + mu.reshape(1, 2) # (A*x)T = xT * AT
#     print(x_y_new)
    
    for xy in x_y_new:
        check_state = tuple(xy.astype(int))
        if(check_state[0] < 0): return False
        if(check_state[0] > env.shape[0]-1): return False
        if(check_state[1] < 0): return False
        if(check_state[1] > env.shape[1]-1): return False
        
        if (is_point_in_collision(env,check_state,7)):
            return False
    
    return True

class GraphRRTBNX:
    
    def __init__(self, start, end, step=75, weight=20):
        self.start = start
        self.end = end
        self.step = step
        self.weight = weight
        
        self.G = nx.DiGraph()
        root_belief = [BeliefNode(np.eye(3)*0.1,np.zeros((3,3)), 0)]
        
        self.G.add_node(0, val = tuple(self.start), belief_nodes = root_belief )
    
    def num_of_vertices(self):
        return len(self.G.nodes)
    
    def distance_angle(self, point1, point2, theta1, theta2):
        return np.linalg.norm(point1 - point2) + self.weight * angle_difference(theta1, theta2)

    def random_init(self):
        key = np.random.choice(range(2), p=[0.1, 0.9])
        if key:
            xnew = int(np.random.uniform(0, 346))
            ynew = int(np.random.uniform(0, 228))
            angle = 0
        
        else:
            xnew, ynew, angle = self.end
            
        return xnew, ynew, angle
    
    def find_nearest(self, random_point):
        min_distance = 1e15
        min_index = None
        
        for index, vert in enumerate(self.G.nodes):
            vert = np.array(self.G.nodes[vert]['val'])
            distance = self.distance_angle(random_point[:2], vert[:2], 
                                    random_point[-1], vert[-1])
            if distance <= min_distance:
                min_distance = distance
                min_index = index
        
        return min_index
    
    @staticmethod
    def calc_radius(n: int, d: int = 3) -> float:
        """
        @n - number of state vertices
        @d - state dimension
        """
        return (np.log(n) / n) ** (1 / d) * 10
    
    def get_neighbors(self,vertex_id):
        return list(self.G.neighbors(vertex_id))

    def near(self, v_new, d = 3):
        n = len(self.G.nodes)

        near_set = []

        radius = self.calc_radius(n, d)
        for vertex in self.G.nodes:
            vertex_coords = np.array(self.G.nodes[vertex]['val'])[:2]
            distance = np.linalg.norm(vertex_coords - v_new[:2])
            if distance <= radius +1e-3:
                near_set.append(vertex)

        return near_set
    
    def find_predecessors(self, node_index):
        return list(self.G.predecessors(node_index))
    
    def steer(self, random_point, min_index):
        
        excess_vector_coords = random_point[:2] - np.array(self.G.nodes[min_index]['val'])[:2]
        normalize = excess_vector_coords / np.linalg.norm(excess_vector_coords)
        
        node_coords = np.array(self.G.nodes[min_index]['val'])[:2]
        delta_coords = node_coords + normalize * self.step
        delta_vector_coords = delta_coords - node_coords
          
        angle = np.array(self.G.nodes[min_index]['val'])[-1]
        new_angle = angle + angle_difference(random_point[-1], angle * np.linalg.norm(delta_vector_coords) / \
                                            np.linalg.norm(excess_vector_coords))
    
        vertex = np.hstack((delta_coords, new_angle))  
        return vertex, new_angle

    def create_plan(self):
        plan = []
        goal_vertices_indices = self.near(self.end[:2])
        
        if goal_vertices_indices is None:
            raise ValueError('Apparently, it is necessary to increase number of iterations.')
            
        best_node = self.G.nodes[goal_vertices_indices[0]]['belief_nodes'][0]
        for vertex in goal_vertices_indices:
            for vertex_node in self.G.nodes[vertex]['belief_nodes']:
                if(self.is_candidate_better(best_node,vertex_node, 1e-3)):
                    best_node = vertex_node
                    print("new node is better")
        current_node = best_node
        vertex_id = self.get_vertex_for_belief_node(best_node)
        plan.append(self.G.nodes[vertex_id]['val'])
        while(current_node != None):
            current_node = current_node.parent
            vertex_id = self.get_vertex_for_belief_node(best_node)
            plan.append(self.G.nodes[vertex_id]['val'])
        print("plan in funct",plan)        
        return plan[::-1]

# think about predict-update relation later
# В текущей реализации начальный state определяет measurement error (r)
    def propagate(self,env,edge, start_belief, dt):
        Sigma = start_belief.Sigma
        states = edge[0]
        actions = edge[1]
        state = states[0]
        Lambda = start_belief.Lambda
        xy_error = 0.0001
        theta_error = 0.0001
        measurement_error = 0.00001
        k = 1e-9
        observable_check_const = 1
#         k = 1e-5
#         state = tuple(state[:2].astype(int)) 
        
        if (is_state_observable(env,tuple(state[:2].astype(int)), observable_check_const)):
            measurement_error = 1e-6
        
        kalman = DynamicModel(xy_error, theta_error,measurement_error, dt, k)
    
        for action in actions:
            kalman.predict(state, action, Sigma)
            kalman.update(Lambda)
            state = kalman.x_updated
            Sigma = kalman.Sigma_updated
            Lambda = kalman.Lambda
            
        # checking collisions only in end_state 
        cov_matrix = Sigma + Lambda 
        cost = start_belief.cost + len(actions)
        if not (uncertainty_collision_check(env,states[-1][:2],cov_matrix[:2,:2], nSigma = 1,n_points = 20)):
            return None
        
        return BeliefNode(Sigma, Lambda, cost, parent = start_belief)    
    
    def is_candidate_better(self,current_node,candidate_node, eps = 1e-3):
        
        is_sigma_better =  np.linalg.norm(current_node.Sigma+np.eye(3)*eps) > np.linalg.norm(candidate_node.Sigma)
        is_lambda_better = np.linalg.norm(current_node.Lambda+np.eye(3)*eps) > np.linalg.norm(candidate_node.Lambda)
        is_cost_better = current_node.cost > candidate_node.cost
#         print(is_sigma_better,is_lambda_better,is_cost_better)
        if (is_sigma_better and is_lambda_better and is_cost_better):
            return True
        else: 
            return False
    
    def get_vertex_for_belief_node(self,belief_node):
        for vertex_id,data in self.G.nodes.data():
            vertex_nodes = data['belief_nodes']
            for node in vertex_nodes:
                if (belief_node == node):
                    return vertex_id
        return None

#!check path function is needed!
    def rttb(self, n_iter=5):
        dt_dynamics = 1e-2
        dt_kalman = 1e-2
        new_vertex_index = -1
        for i in range(n_iter):
            if not i % 30:
                print('Iteration:', self.num_of_vertices())
            belief_nodes_to_explore = []
            random_point = self.random_init()
            
            # Find index of the closest node
            min_index = self.find_nearest(random_point)
            
            # Steer new coords
            vertex, new_angle = self.steer(random_point, min_index)  
                        
            path = connect(self.G.nodes[min_index]['val'], vertex, dt_dynamics, plot = False)
            
            propagated = False
            for belief_node in self.G.nodes[min_index]['belief_nodes']:
                candidate = self.propagate(env,path, belief_node, dt_kalman)
                if(candidate != None):
                    propagated = True
                    new_vertex_index = self.num_of_vertices()
#                     print("belief_node.cost", candidate.cost,  belief_node.cost)
#                     
#                     candidate.parent = belief_node
                    self.G.add_node(new_vertex_index, val=tuple(vertex),belief_nodes= [candidate])
                    self.G.add_edge(min_index, new_vertex_index)
                    belief_nodes_to_explore += self.G.nodes[min_index]['belief_nodes'] 
                    
                    break
#                 cov_matrix = candidate.Sigma + candidate.Lambda 
#                 if (uncertainty_collision_check(env,vertex[:2],cov_matrix[:2,:2], nSigma = 1,n_points = 50)):
            if (not propagated):
#                 print(min_index,len(self.G.nodes[min_index]['belief_nodes']),i,propagated)
                continue
            
            near_vertices_indices = self.near(vertex)
#             print("near_vertices_indices",near_vertices_indices,
#                   "new_ver", self.num_of_vertices()-1, "min_ver", min_index)
            try:
                near_vertices_indices.remove(min_index)
            except Exception as e:
                pass

            for index in near_vertices_indices:
                if index != new_vertex_index:
                    self.G.add_edge(index, new_vertex_index)
                    belief_nodes_to_explore += self.G.nodes[index]['belief_nodes'] 
            
            while(belief_nodes_to_explore):
                current_node = belief_nodes_to_explore.pop()
                vertex_id = self.get_vertex_for_belief_node(current_node)
                for neigh_vertex in self.get_neighbors(vertex_id):
                    path_from_node_to_neigh = connect(self.G.nodes[vertex_id]['val'],
                                                      self.G.nodes[neigh_vertex]['val'],
                                                      dt_dynamics, plot = False)
                    candidate = self.propagate(env,path_from_node_to_neigh, current_node, dt_kalman)
                    if(candidate != None):
                        for vertex_node in self.G.nodes[neigh_vertex]['belief_nodes']:
#                             print("vertex_node", type(vertex_node))
                            if(self.is_candidate_better(vertex_node,candidate,0)):
                                self.G.nodes[neigh_vertex]['belief_nodes'].append(candidate)
    #                                 print(self.G.nodes[neigh_vertex]['belief_nodes'])
    #                                 if (len(self.G.nodes[neigh_vertex]['belief_nodes'])> 5):
    #                                     print("overloaded node")
                                belief_nodes_to_explore +=  [candidate]
                                print("new node is better")
    #                             else: 
    #                                 print("new node is worse")


In [157]:
# env, obj, start, stop = load_data('PS2_data.npz')

initial_point = np.array([200, 20, 0])
finish_point = np.array([50, 163, 0])
weight = 20
step = 5
g = GraphRRTBNX(initial_point, finish_point, weight=weight, step = step)

g.rttb(300)

Iteration: 1
Iteration: 11
Iteration: 21
Iteration: 31
Iteration: 41
Iteration: 51
Iteration: 61
Iteration: 71
Iteration: 76
Iteration: 86
Iteration: 95
Iteration: 103
Iteration: 109
Iteration: 118
Iteration: 128
Iteration: 137
Iteration: 147
Iteration: 155
Iteration: 165
Iteration: 174
Iteration: 182
Iteration: 191
Iteration: 201
Iteration: 211
Iteration: 221
Iteration: 231
Iteration: 239
Iteration: 247
Iteration: 257
Iteration: 266


In [158]:
point = finish_point[:2]
def near(G, v_new, d = 3):
    n = len(G.nodes)

    near_set = []

    radius = 5
    for vertex in G.nodes:
        vertex_coords = np.array(G.nodes[vertex]['val'])[:2]
        distance = np.linalg.norm(vertex_coords - v_new[:2])
        if distance <= radius +1e-3:
            near_set.append(vertex)

    return near_set
# Near vertices indices
print(g.num_of_vertices())
print(near(g.G,point))

print(g.create_plan())

276
[127, 128, 131, 144, 148, 154, 157, 159, 162, 164, 165, 176, 183, 189, 228, 231, 244, 265]
plan in funct [(50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675495, 161.7892664419323, 0.0), (50.760552895675

In [ ]:
# plt.figure(figsize=(12, 8))
# nx.draw_networkx(g.G)


In [ ]:
            
#                 print("self.get_neighbors(vertex_id)",self.get_neighbors(vertex_id),vertex_id)
#                 for()
#             print(belief_nodes_to_explore)
#             print(len(belief_nodes_to_explore))
            
    
    
#                 if check_collision_obj_image(
#                     env, 
#                     rotate_image(obj, new_angle), 
#                     np.array(self.G.nodes[min_index]['val'])[:2].astype(int),
#                     vertex[:2].astype(int),
#                     threshold=12 # Empiristically
#                 )


       
def sure_path_exists(self):
    vertices = [self.G.nodes[i]['val'] for i in range(len(self.G.nodes))]
    indices = distance_angle_all(np.array(vertices), self.end).flatten().argsort()
    for index in indices:
        if not check_collision_obj_image(
            env, 
            rotate_image(obj, vertices[index][-1]),
            np.array(vertices[index])[:2].astype(int),
            self.end[:2].astype(int),
            threshold=12 # Empiristically
        ):
            return index


# NEAR

# WHO OWNS

In [ ]:
g.find_predecessors(36)

In [ ]:
def trim_zeros(array):
    for axis in [0, 1]:
        mask = ~(array == 0).all(axis=axis)
        inv_mask = mask[::-1]
        start_idx = np.argmax(mask == True)
        end_idx = len(inv_mask) - np.argmax(inv_mask == True)
        if axis:
            array = array[start_idx:end_idx, :]
        else:
            array = array[:, start_idx:end_idx]
            
    return array


In [ ]:
coords = np.array([200, 50])
ellipse = create_ellipse(5, 6, -40)

plt.figure(figsize=(12, 8))
plt.imshow(merge_images(env, ellipse, coords))

In [ ]:
initial_coords = np.array([200, 50])
new_coords = np.array([50, 50])

fig, ax = plt.subplots(1, 2)
plt.figure(figsize=(12, 8))

ax[0].imshow(merge_images(env, ellipse, initial_coords))
ax[1].imshow(merge_images(env, ellipse, new_coords))

print(check_collision_obj_image(
    env, 
    obj, 
    initial_coords,
    new_coords,
    threshold=12
))

In [ ]:
a = (1,2)
a[0]

# GRAPH RRT 

In [ ]:
def distance_angle_all(point1, point2):
    return np.linalg.norm(point1[:, :2] - point2[:2], axis=1)[:, None] \
                + 20 * angle_difference(point1[:, -1], finish_point[-1])[:, None]

class GraphRRT:
    
    def __init__(self, start, end, step=75, weight=20):
        self.start = start
        self.end = end
        self.step = step
        self.weight = weight
        
        self.G = nx.DiGraph()
        self.G.add_node(0, val = tuple(self.start))
    
    def num_of_vertices(self):
        return len(self.G.nodes)
    
    def distance_angle(self, point1, point2, theta1, theta2):
        return np.linalg.norm(point1 - point2) + self.weight * angle_difference(theta1, theta2)

    def random_init(self):
        
        key = np.random.choice(range(2), p=[0.1, 0.9])
        if key:
            xnew = int(np.random.uniform(0, 346))
            ynew = int(np.random.uniform(0, 228))
            angle = 0
        
        else:
            xnew, ynew, angle = self.end
            
        return xnew, ynew, angle
    
    def find_nearest(self, random_point):
        min_distance = 1e15
        min_index = None
        
        for index, vert in enumerate(self.G.nodes):
            vert = np.array(self.G.nodes[vert]['val'])
            distance = self.distance_angle(random_point[:2], vert[:2], 
                                    random_point[-1], vert[-1])
            if distance <= min_distance:
                min_distance = distance
                min_index = index
        
        return min_index
    
    def steer(self, random_point, min_index):
        
        excess_vector_coords = random_point[:2] - np.array(self.G.nodes[min_index]['val'])[:2]
        normalize = excess_vector_coords / np.linalg.norm(excess_vector_coords)
        
        node_coords = np.array(self.G.nodes[min_index]['val'])[:2]
        delta_coords = node_coords + normalize * self.step
        delta_vector_coords = delta_coords - node_coords
          
        angle = np.array(self.G.nodes[min_index]['val'])[-1]
        new_angle = angle + angle_difference(random_point[-1], angle * np.linalg.norm(delta_vector_coords) / \
                                            np.linalg.norm(excess_vector_coords))
    
        vertex = np.hstack((delta_coords, new_angle))  
        return vertex, new_angle

    def sure_path_exists(self):
        vertices = [self.G.nodes[i]['val'] for i in range(len(self.G.nodes))]
        indices = distance_angle_all(np.array(vertices), self.end).flatten().argsort()
        for index in indices:
            if not check_collision_obj_image(
                env, 
                rotate_image(obj, vertices[index][-1]),
                np.array(vertices[index])[:2].astype(int),
                self.end[:2].astype(int),
                12
            ):
                return index
            
    def create_plan(self):
        point = self.sure_path_exists()
        plan = []
        
        if point is None:
            raise ValueError('Apparently, it is necessary to increase number of iterations.')

        while True:
            current_node = self.G.nodes[point]['val']
            plan.append(current_node)
            try:
                point = list(self.G.predecessors(point))[0]

            except IndexError:
                break
                
        return plan[::-1]

    def rtt(self, n_iter=5):
        
        for i in range(n_iter):
            
            if not i % 200:
                print('Iteration:', i)

            random_point = self.random_init()
            
            # Find index of the closest node
            min_index = self.find_nearest(random_point)
            
            # Steer new coords
            vertex, new_angle = self.steer(random_point, min_index)   
            
            if check_collision_obj_image(
                env, 
                rotate_image(obj, new_angle), 
                np.array(self.G.nodes[min_index]['val'])[:2].astype(int),
                vertex[:2].astype(int),
                12
            ):
                continue
                
            new_vertex_index = self.num_of_vertices()
            self.G.add_node(new_vertex_index, val=tuple(vertex))
            self.G.add_edge(min_index, new_vertex_index)
            
            

In [ ]:
initial_point = np.array([200, 20, 0])
finish_point = np.array([50, 163, 0])
weight = 20 # Не нужна, ибо угол всегда 0 (оставил на всякий случай)
step = 5
g = GraphRRT(initial_point, finish_point, weight=weight, step=step)

g.rtt(200)

plan = g.create_plan()
# plotting_results(env, obj, plan, weight=weight, step=step)

In [ ]:
g.G.[(0, 1)]

# ЧТОБЫ КАРТИНКУ ПОСТРОИТЬ

In [ ]:
from typing import Union

def calc_radius(n: int, d: int, coef: Union[int, float]) -> float:
    """
    @n - number of state vertices
    @d - state dimension
    @coef - size of circles
    """
    return (np.log(n) / n) ** (1 / d) * coef

def near(V, v_new, d=3):
    """
    @V - set of vertices
    @v_new - 
    
    """
    n = len(V)
    
    near_set = []
    
    radius = calc_radius(n, d)
    for vertex in V:
        distance = np.linalg.norm(vertex - v_new)
        print('Distance', distance)
        print('Radius', radius)
        if distance <= radius:
            near_set.append(vertex)
            
    return near_set

plt.figure(figsize=(12, 8))

xaxis = [x[1] for x in plan]
yaxis = [x[0] for x in plan]

verbose_step = 10
radius = calc_radius(2, 3, 30)

for i in range(len(plan)):
    if not i % verbose_step - 1:
        circle = plt.Circle((xaxis[i], yaxis[i]), radius, color='magenta', fill=False, linewidth=3)
        plt.gca().add_patch(circle)

plt.plot(xaxis, yaxis, color='white', linewidth=3)
plt.imshow(env)

plt.axis('off')

plt.show()

# plt.savefig('rrt.png', bbox_inches='tight', pad_inches=0)